In [4]:
# handle names for expressions - change expression names within expression
# go over classes and find invalid assumptions

In [5]:
#hypster prepare
#Check if any parameter is HpExpression. if not: return the original object
#If there is an HpExpression in it
    #Save the original call for class/function
    #Save args and kwargs as separate dicts

In [6]:
import fastai2
from fastai2.tabular.all import *
from fastai2.metrics import *

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
from copy import deepcopy

In [9]:
import optuna

In [10]:
SEED = 42

# HyPSTER Classes

In [11]:
from inspect import signature, Parameter
import functools

def auto_assign(func):
    # Signature:
    sig = signature(func)
    for name, param in sig.parameters.items():
        if param.kind in (Parameter.VAR_POSITIONAL, Parameter.VAR_KEYWORD):
            raise RuntimeError('Unable to auto assign if *args or **kwargs in signature.')
    # Wrapper:
    @functools.wraps(func)
    def wrapper(self, *args, **kwargs):
        for i, (name, param) in enumerate(sig.parameters.items()):
            # Skip 'self' param:
            if i == 0: continue
            # Search value in args, kwargs or defaults:
            if i - 1 < len(args):
                val = args[i - 1]
            elif name in kwargs:
                val = kwargs[name]
            else:
                val = param.default
            setattr(self, name, val)
        func(self, *args, **kwargs)
    return wrapper

In [12]:
def sample_hp(hp, trial): return hp if not isinstance(hp, HpExpression) else hp.sample(trial)

In [13]:
class HpExpression(object):
    def __init__(self, exp1, exp2):
        self.exp1 = exp1; self.exp2 = exp2    
    
    def sample(self, trial): raise NotImplementedError
    
    def get_name(self):
        if self.name is not None: return self.name

        name = ""
        if self.exp1 is not None and isinstance(self.exp1, HpExpression) and hasattr(self.exp1, "name"):
            name += self.exp1.name
        if self.exp2 is not None and isinstance(self.exp2, HpExpression) and hasattr(self.exp2, "name"):
            if len(name) > 0:
                name +=  "_"
            name += self.exp2.name
        self.name = name
        return self.name
        #TODO: refactor
        
    def __add__(self, other):  return AddExpression(self, other)
    def __radd__(self, other): return AddExpression(other, self)
    def __sub__(self, other):  return SubExpression(self, other)
    def __rsub__(self, other): return SubExpression(other, self)
    def __mul__(self, other):  return MulExpression(self, other)
    def __rmul__(self, other): return MulExpression(other, self)
    def __div__(self, other):  return DivExpression(self, other)
    def __rdiv__(self, other): return DivExpression(other, self)
    def __pow__(self, other):  return PowExpression(self, other)
    def __rpow__(self, other): return PowExpression(other, self)

In [14]:
class SubExpression(HpExpression):
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 - exp2

In [15]:
class AddExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 + exp2

In [16]:
class MulExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 * exp2

In [17]:
class DivExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 / exp2

In [18]:
class PowExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 ** exp2

In [19]:
#TODO: add round, int, floor etc...
#TODO: add Brackets () etc...?

In [20]:
class HpFloat(HpExpression):
    @auto_assign
    def __init__(self, name, low, high, log=False, step=None):
        self.result = None
        #TODO: check what's up with log and step
        #TODO: move result to HpExpression?
        
    def sample(self, trial): 
        self.result = ifnone(self.result, trial.suggest_float(self.name, self.low, self.high))
        return self.result
    
    #TODO: warn if log=True & step is not None
    #TODO: check what is the "*" in the function definition

In [21]:
class HpInt(HpExpression):
    @auto_assign
    def __init__(self, name, low, high, step=1):
        self.result = None
    
    def sample(self, trial):
        self.result = ifnone(self.result, trial.suggest_int(self.name, self.low, self.high, self.step))
        return self.result

In [22]:
class HpCategorical(HpExpression):
    @auto_assign
    def __init__(self, name, choices): 
        self.result = None
    
    def sample(self, trial): 
        if self.result is not None:
            return self.result
        
        choices           = self.choices
        name              = self.name
        optuna_valid_cats = ["str", "int", "float", "bool"] #TODO: add more + move to global area
        
        if any([type(choice) not in optuna_valid_cats for choice in self.choices]):
            self.items_names      = [choice.__name__ for choice in choices]
            self.dict_items_names = dict(zip(self.items_names, choices))
            chosen_hp             = trial.suggest_categorical(name, self.dict_keys)
            self.result           = self.dict_items[chosen_hp]
            #TODO: add items dict
            #TODO: add check for "choice.__name__"
        else:
            self.result = trial.suggest_categorical(name, choices)
        return self.result

In [23]:
class HpList(HpExpression):
    @auto_assign
    def __init__(self, name, min_len, max_len, hp, same_value=False): pass        
    
    def sample(self, trial):
        lst_len = trial.suggest_int(self.name, self.min_len, self.max_len)
        lst = []
        if (self.same_value) or (not isinstance(self.hp, HpExpression)):
            lst = [sample_hp(self.hp, trial)] * lst_len
        else:
            for i in range(lst_len):
                hp = deepcopy(self.hp)
                hp.result = None
                hp.name = f"{hp.get_name()}_{i+1}"
                result = sample_hp(hp, trial)
                lst.append(result)
                #TODO keep self.result?
        return lst

In [24]:
class HpIterable(HpExpression):
    def __init__(self, name, iterable): 
        self.name = name
        self.iterable = iterable

    def _sample_list(self, trial, lst):  return [sample_hp(item, trial) for item in lst]
    def _sample_dict(self, trial, dct):  return {key : sample_hp(value, trial) for key, value in dct.items()}
    def _sample_tuple(self, trial, tup): return (*self._sample_list(trial, tup), )
    def _sample_L(self, trial, l):       return L(self._sample_list(trial, l))
    
    def sample(self, trial):
        if   isinstance(self.iterable, dict):   return self._sample_dict(trial, self.iterable)
        elif isinstance(self.iterable, list):   return self._sample_list(trial, self.iterable)
        elif isinstance(self.iterable, L):      return self._sample_L(trial, self.iterable)
        elif isinstance(self.iterable, tuple):  return self._sample_tuple(trial, self.iterable)
        else:                                   print("Error: unknown Iterable!")
        return

In [25]:
class HpBool(HpCategorical):
    def __init__(self, name):
        super().__init__(name, choices=[False, True])

In [26]:
class HpToggle(HpBool):
    def __init__(self, hp): return
    def sample(self, trial): return trial.suggest_categorical(f"toggle_{hp.name}", [False, True]) 
    #TODO: fix hp.name

In [27]:
x = HpInt("start_mom", 2, 10)

In [28]:
y = HpInt("layer_size", 50, 300, 50)
y = HpBool("booli!")
x = HpList("n_layers", 1, 5, y, same_value=False)

In [29]:
optuna.logging.set_verbosity(0)
pruner = optuna.pruners.NopPruner()
study = optuna.create_study(direction="maximize", pruner=pruner)

In [30]:
def objective(trial):
    print(x.sample(trial))
    #print(y.sample(trial))
    return 1.0

In [31]:
study.optimize(objective, n_trials=5, timeout=600)

[W 2020-04-28 21:54:02,598] Setting status of trial#0 as TrialState.FAIL because of the following error: AttributeError("'bool' object has no attribute '__name__'")
Traceback (most recent call last):
  File "/Users/giladrubin/anaconda3/lib/python3.7/site-packages/optuna/study.py", line 677, in _run_trial
    result = func(trial)
  File "<ipython-input-30-7889c85e27c8>", line 2, in objective
    print(x.sample(trial))
  File "<ipython-input-23-363f3472c934>", line 15, in sample
    result = sample_hp(hp, trial)
  File "<ipython-input-12-79d0fd047f55>", line 1, in sample_hp
    def sample_hp(hp, trial): return hp if not isinstance(hp, HpExpression) else hp.sample(trial)
  File "<ipython-input-22-f3f1fb5fce54>", line 15, in sample
    self.items_names      = [choice.__name__ for choice in choices]
  File "<ipython-input-22-f3f1fb5fce54>", line 15, in <listcomp>
    self.items_names      = [choice.__name__ for choice in choices]
AttributeError: 'bool' object has no attribute '__name__'


AttributeError: 'bool' object has no attribute '__name__'

In [3]:
study.trials_dataframe()

NameError: name 'study' is not defined

In [ ]:
#TODO consider adding "name" into HpExpression

In [ ]:
#HpFunc

In [ ]:
#HpTuple (like list)
#HpIterable (?)

In [ ]:
#TODO: support expressions like 5 * HpInt(...)
#TODO: support expressions like HpInt(...) * HpFloat(...)

# Read Data

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

In [ ]:
df = pd.read_csv(path/'adult.csv')
df.head()

In [ ]:
df = df.sample(frac=0.3)

In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']

In [ ]:
cont_names = ['age', 'fnlwgt', 'education-num']

In [ ]:
dep_var = "salary"

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.6, 
                                     random_state=SEED, 
                                     stratify=df[dep_var])

# Preprocessing

In [ ]:
cat = Categorify()

In [ ]:
imp = FillMissing(fill_strategy=FillStrategy.mode, 
                  add_col=True)

In [ ]:
norm = Normalize()

In [ ]:
procs = [cat, imp, norm]

# DataBunch

In [ ]:
to = TabularPandas(train_df, 
                   y_block = CategoryBlock(), 
                   y_names = dep_var,
                   splits = RandomSplitter()(range_of(train_df)),
                   cat_names = cat_names,
                   cont_names = cont_names,
                   procs = procs)

In [ ]:
dls = to.dataloaders(batch_size=32)

# Learner

In [ ]:
cbs = [TrackerCallback(monitor="roc_auc_score"), ReduceLROnPlateau("roc_auc_score", patience=3)]

In [ ]:
class HypsterBase(object):
    def __init__(self): return

In [ ]:
class HypsterTabularLearner(HypsterBase):
    def __init__(self, dls, **kwargs):
        self.dls = dls
        self.__dict__.update(kwargs)
        self.func = raw_tabular_learner

In [ ]:
raw_tabular_learner = fastai2.tabular.learner.tabular_learner

In [ ]:
hypster_types = (HypsterBase, HpExpression)

In [ ]:
data_structures = (set, list, dict, L)

In [ ]:
def _contains_hypster(x):
    if not isinstance(x, data_structures):
        x = [x]
    elif isinstance(x, dict):
        x = x.values()
        
    for item in x:
        if isinstance(item, data_structures):
            if _iterable_contains_hypster(item):
                return True
        else:
            if isinstance(item, hypster_types):
                return True
    return False

In [ ]:
@delegates(raw_tabular_learner)
def tabular_learner(dls, **kwargs):
    kwargs["dls"] = dls
    for key, value in kwargs.items():
        if _contains_hypster(value):
            return HypsterTabularLearner(**kwargs)
    return raw_tabular_learner(**kwargs)

In [ ]:
#def tabular_learner():
    #put arguments in func definition
    #add state to learner = superposition or not
    #add trial/study
    #check if opt_func is type list
        #if so - randomly choose one
    #call function

In [ ]:
cbs = [TrackerCallback(monitor="roc_auc_score"), 
       HpToggle(ReduceLROnPlateau("roc_auc_score", patience=HpInt("patience", 1, 5)))]

In [ ]:
cbs = [TrackerCallback(monitor="roc_auc_score"), 
       ReduceLROnPlateau("roc_auc_score", patience=1)
      ]

In [ ]:
start_mom = HpFloat("start_mom", 0.85, 0.99)

In [ ]:
learn = tabular_learner(dls=dls, 
                        metrics=RocAuc(),
                        layers=HpList("layers", 1, 3, 50 * HpInt("layer_size", 1, 6), same_value=False),
                        opt_func=HpCategorical("optimizer", [Adam, SGD, QHAdam]),
                        cbs=cbs,
                        moms=(start_mom, start_mom-0.1, start_mom), 
                        #wd_bn_bias=HpBool("wd_bn_bias"), 
                        )

# Optuna

In [ ]:
class HypsterExperiment(HypsterBase):
    def __init__(self, learner):
        self.learner = learner
    def fit(self, n_trials):
        self.study = study_fit(self.learner, n_trials)        

In [ ]:
from inspect import signature

In [ ]:
def get_func_args(sig):
    args = []
    for arg, value in sig.parameters.items():
        if not "=" in str(value):
            args.append(arg)
    return args

In [ ]:
def pop_move_dict(dict1, keys):
    lst = []
    for key in list(dict1):
        if key in keys:
            lst.append(dict1.pop(key))
    return lst

In [ ]:
class Objective(object):
    def __init__(self, learner):
        self.learn = learner
    def __call__(self, trial):
        learner = deepcopy(self.learn)
        func = learner.__dict__.pop("func")
        actual_dict = {}
        for key, value in learner.__dict__.items():
            if _contains_hypster(value):
                if isinstance(value, data_structures):
                    
                else:
                    actual_dict[key] = value.sample(trial)
            else:
                actual_dict[key] = value
        
        sig = signature(func)
        func_args = get_func_args(sig)
        args = pop_move_dict(actual_dict, func_args)
        print(args)
        print("\n")
        print(actual_dict)
        actual_learner = func(*args, **actual_dict)
        actual_learner.fit_one_cycle(2)
        print(actual_learner.cbs)
        return actual_learner.cbs[3].best

In [ ]:
def study_fit(learner, n_trials):
    optuna.logging.set_verbosity(0)
    pruner = optuna.pruners.NopPruner()
    study = optuna.create_study(direction="maximize", pruner=pruner)
    objective = Objective(learner)
    study.optimize(objective, n_trials=n_trials, timeout=600)
    return study

In [ ]:
clf = HypsterExperiment(learn)

In [ ]:
clf.fit(n_trials=2)

In [ ]:
clf.study.trials_dataframe()

In [ ]:
#learn = ...
hps = HypsterExperiment(learn, learn.fit_one_cycle, n_trials=3) 
hps.fit()

In [ ]:
import optuna

In [ ]:
EPOCHS = 2

In [ ]:
def objective(trial):
    #Q: How do I define fill value?
    cat = Categorify()
    fillstraFillStrategy.mode
    imp = FillMissing(fill_strategy=fill_strategy,
                      add_col=True)

    norm = Normalize(mean=5)
    procs = [cat, imp, norm]

    # DataBunch
    to = TabularPandas(train_df, 
                       y_block = CategoryBlock(), 
                       y_names = dep_var,
                       splits = RandomSplitter()(range_of(train_df)),
                       cat_names = cat_names,
                       cont_names = cont_names,
                       procs = procs)

    dls = to.dataloaders(batch_size=512)
    
    n_layers = trial.suggest_int("n_layers", 2, 5)
    layer_sizes = L()
    
    for i in range(n_layers):
        layer_size = trial.suggest_int("layer_size_{}".format(i), 1, 10)
        layer_sizes.append(50 * layer_size)

    learn = tabular_learner(dls, metrics=RocAuc(),
                            layers=layer_sizes, 
                            #emb_szs=[], 
                            #loss_func,
                            #opt_func, 
                            cbs=cbs,
                            #lr=0.001, moms=(0.95, 0.85, 0.95)
                            #wd=None, wd_bn_bias=False, train_bn=True
                           )
    
    learn.fit_flat_cos(EPOCHS)

    return learn.cbs[3].best

In [ ]:
optuna.logging.set_verbosity(0)
pruner = optuna.pruners.NopPruner()
study = optuna.create_study(direction="maximize", pruner=pruner)
study.optimize(objective, n_trials=5, timeout=600)

In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))

In [ ]:
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))

In [ ]:
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

# Test Results

In [ ]:
test_dl = learn.dls.test_dl(test_df.drop(['salary'], axis=1))

In [ ]:
probs = learn.get_preds(dl=test_dl)

In [ ]:
probs = probs[0][:,1]

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
scorer = RocAuc()

In [ ]:
roc_auc_score(test_df[dep_var], probs)

# Desired API

In [ ]:
import numpy as np

## Preprocessing

In [ ]:
fill = HpInt(start_range=1, end_range=100)

In [ ]:
fill_strategy = HpOptions("fill_strategy", 
                          [FillStrategy.mode, 
                           FillStrategy.median, 
                           FillStrategy.constant(5, fill)])

In [ ]:
# or less preferred

In [ ]:
fill_dict = {"mode" : FillStrategy.mode, "median" : FillStrategy.median, "constant" : FillStrategy.constant(5, fill)}
fill_strategy = HpOptions("fill_strategy", ["mode", "median", "constant"], fill_dict)

In [ ]:
add_col = HpBool("missing_col_bool")

In [ ]:
imp = FillMissing(fill_strategy=fill_strategy, add_col=add_col)

#### Option A

In [ ]:
norm = Normalize(mean=HpFloat("norm_mean", start=2, end=10, dist="uniform"))

In [ ]:
procs = [Categorify, imp, HpToggle(norm)]

#### Option B

In [ ]:
norm = HpToggle(Normalize(mean=HpFloat(start=2, end=10, dist="uniform")))

In [ ]:
procs = [Categorify, imp, norm]

## DataBunch

In [ ]:
to = TabularPandas(train_df, 
                   y_block = CategoryBlock(), 
                   y_names = dep_var,
                   splits = RandomSplitter()(range_of(train_df)),
                   cat_names = cat_names,
                   cont_names = cont_names,
                   procs = procs)

#### Option A

In [ ]:
bs_pow = HpInt(0, 8)

In [ ]:
dls = to.dataloaders(batch_size=2**bs_pow)
#or
dls = to.dataloaders(batch_size=HpConst(2)**bs_pow)

#### Option B

In [ ]:
dls = to.dataloaders(batch_size=HpBatchSizeFinder(...))

## Learner

In [ ]:
from fastai2.metrics import *

In [ ]:
cbs = [TrackerCallback(monitor="roc_auc_score")]

### #Layers + Layer Sizes

#### Option A

In [ ]:
layer_size_hp = HpFuncInt(func=np.multiply, base_value=50, min_int=1, max_int=7)
layers = HpVarList(min_len=1, max_len=5, layer_size_hp)

#### Option B

In [ ]:
layer_size_hp = HpInt(min=1, max=7)
layers = HpVarList(min_len=1, max_len=5, value=50 * layer_size_hp) 
#TODO: think of how to distinguish between same value for all items in list and different ones?

### Optimizer

In [ ]:
opt_dict = {"SGD" : SGD, "ADAM" : Adam, "LAMB" : fastai2.optimizer.Lamb()}

In [ ]:
optimizer = HpOptions(["SGD", "ADAM", "LAMB"], opt_dict)

In [ ]:
if optimizer == "SGD":
    sqr_mom = Adam()
    Lamb(sqr_mom)

#### Option B

In [ ]:
def opt_name_to_opt(name): if name.containts("SGD") return SGD else Adam

In [ ]:
optimizer = HpOptions(["SGD", "ADAM", "LAMB"], opt_name_to_opt)

#### Option C

In [ ]:
optimizer = HpOptions([SGD, Adam, Lamb])

### Init Learner

In [ ]:
learn = tabular_learner(dls, metrics=RocAuc(),
                        layers=layers
                        
                        #loss_func,
                        opt_func = opt_func
                        cbs=cbs,
                        #moms=(0.95, 0.85, 0.95)
                        #wd=None, wd_bn_bias=False, train_bn=True
                        #emb_szs=[],
                       )

### LR Finder

In [ ]:
lr = HpLrFinder(finder_type="fastai", which="steep", kwargs=...)

In [ ]:
learn.fit_flat_cos(3, lr=lr)